In [62]:

selectedPrime = 149

a = 1
b = 1

R.<x,y> = PolynomialRing(GF(selectedPrime),2,order='invlex')
Q.<x,y> = QuotientRing(R,R.ideal(y**2 - x**3 -a*x -b))


fPolycache = {}

def getFPoly(m):
    toReturn = None
    if m in fPolyCache:
        return fPolyCache[m]

    if m == 0:
        toReturn = 0
    elif m == 1 or m == 2:
        toReturn = 1
    elif m == 3:
        toReturn = 3*x**4+6*a*x**2+12*b*x-a**2
    elif m == 4:
        toReturn = 2*(x**6 + 5*a*x**4 + 20*b*x**3 - 5*a*a*x**2 - 4*a*b*x - 8*b**2 - a**3)
    else:
        if m % 2 == 0:
            mHalf = m // 2
            toReturn = getFPoly(mHalf) * (getFPoly(mHalf + 2) * getFPoly(mHalf - 1)**2 - getFPoly(mHalf - 2) * getFPoly(mHalf + 1)**2)
        else:
            mHalf = (m-1)//2
            if mHalf % 2 == 0:
                toReturn = (16*(x**3+a*x+b)**2)*getFPoly(mHalf+2)*getFPoly(mHalf)**3 - getFPoly(mHalf-1)*getFPoly(mHalf+1)**3
            else:
                toReturn = getFPoly(mHalf+2)*getFPoly(mHalf)**3 -(16*(x**3+a*x+b)**2)*getFPoly(mHalf-1)*getFPoly(mHalf+1)**3
    fPolyCache[m] = toReturn
    return fPolyCache[m]

def getSPoly(m):
    ql = selectedPrime % m
    if ql == 1:
        return x**(selectedPrime**2)-x
    if ql % 2 == 0:
        return 4*(x**(selectedPrime**2)-x)*(x**3+a*x+b)*getFPoly(ql)**2+getFPoly(ql-1)*getFPoly(ql+1)
    else:
        return (x**(selectedPrime**2)-x)*getFPoly(ql)**2+(4*(x**3+a*x+b))*getFPoly(ql-1)*getFPoly(ql+1)

def equalx(l):
    return 0
    
def nonequalx(l, tau):
    return 0
def schoff():
    B = 2
    l = 2
    curvePoly = x**3+a*x+b
    cycloPoly = x**selectedPrime - x
    if Q(R(curvePoly).gcd(R(cycloPoly))) == 1:
        tau = 1
    else:
        tau = 0
    residues = [tau]
    moduli = [2]
    
    while B < 4*sqrt(selectedPrime):
        l = l.next_prime()
        B = B*l
        fl = getFPoly(l)
        sl = getSPoly(l)
        sl = R(sl) % R(fl)
        if Q(R(fl).gcd(R(sl))) == 1:
            tau = equalx(l)
        else:
            r = None
            tau = 0
            while r != 0:
                tau += 1
                r = nonequalx(l, tau)
            if r == -1:
                tau = -tau
        residues.append(tau)
        moduli.append(l)
    print(residues)
    print(moduli)
    return crt(residues, moduli)

schoff()


[0, 0, 1, 1]
[2, 3, 5, 7]


36